In [21]:
import astropy.io.fits as fits
import numpy as np
from scipy.integrate import quad
import scipy.integrate as integrate
from scipy.optimize import curve_fit
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, join
import os
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import halomod as hm
import hmf
import treecorr
import DarkVerse as dv


%matplotlib inline
plt.rc('font',**{'family':'serif','size':14})
#plt.rc('text', usetex=True)
home_dir = os.path.expanduser('~')
directory_SN_X3 = "/home/astro/manasoi7/Master_Thesis/Results/SN-X3"
directory_SN_C3 = "/home/astro/manasoi7/Master_Thesis/Results/SN-C3"
directory_SN_E2 = "/home/astro/manasoi7/Master_Thesis/Results/SN-E2" 
directory_COSMOS = "/home/astro/manasoi7/Master_Thesis/Results/COSMOS" 
res = os.path.join(home_dir, 'Master_Thesis', 'Results')




home_dir = os.path.expanduser('~')
thesis_path = os.path.join(home_dir, 'Master_Thesis', 'DATA')



# Catalog
fits_file_path = os.path.join(thesis_path, "Y3_deep_fields_DB_wKNN_cat_SN-C3_zm.fits")  
t= Table.read(fits_file_path)
masked_file_path = os.path.join(thesis_path, "SN-C3_masked_cat.fits")  
t3= Table.read(masked_file_path)

# Join in one table

t=join(t,t3,keys='id') 
t.rename_column('ra_1','ra')
t.rename_column('dec_1','dec')


# Load Randoms
fits_random = os.path.join(thesis_path, "SN-C3_randoms_ugriz_trim_video.fits") 
t2 = Table.read(fits_random)
randoms = t2  # Random catalog

# Define Correlation Function Configuration
config = {
    'min_sep': 0.003, 
    'max_sep': 1.78,
    'bin_size': 0.1,
    'sep_units': 'degrees',
    'var_method': 'shot'   # if using the internal 'var_method': 'bootstrap' of treecrorr, but then it takes a long time to run
}






In [22]:
stellar_masses = []
halo_masses = []

# Define redshift range and stellar mass bins
z_min, z_max = 0.2, 0.4
SM_bins = [ (8.5, 12.5), (8.9, 12.5),(9.1, 12.5),(9.5, 12.5), (9.9, 12.5), (10.1, 12.5), (10.5, 12.5), (10.9, 12.5), (11.5, 12.5)]


# Assume you have `t` and `randoms` from one field loaded already
# (Or loop over fields like before if needed)

for SM_min, SM_max in SM_bins:
    print(f"\n🔹 Processing bin: logSM ∈ [{SM_min}, {SM_max}]")

    sub = dv.Selection(
        catalog=t,
        randoms=randoms,
        z_min=z_min,
        z_max=z_max,
        SM_min=SM_min,
        SM_max=SM_max,
        config=config
    )

    try:
        # Optional: custom initial guess, can be adjusted
        hod_params, pcov = sub.fit_hod(p0=[12.3 * 1e-7, 13.5, 1.0])
        results = sub.get_results()

        logM_min, logM_1, alpha = results['hod_params']

        logM_star = SM_min
        
        logM_halo = logM_min * 1e7  # Assuming it's scaled
        
        M_star = 10**logM_star
        M_halo = 10**logM_halo

        print(f"✅ HOD fit success: logM_star = {logM_star:.2f}, logM_min = {logM_min*1e7:.2f}")

        stellar_masses_SN_C3.append(M_star)
        halo_masses_SN_C3.append(M_halo)

    except Exception as e:
        print(f"❌ HOD fitting failed for bin {SM_min}–{SM_max}: {e}")



🔹 Processing bin: logSM ∈ [8.5, 12.5]


KeyboardInterrupt: 

In [ ]:

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter( halo_masses_SN_C3, stellar_masses_SN_C3, marker='o', linestyle='-', color='purple', label= 'SN-C3')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r'Stellar Mass $M_*$ [$M_\odot$]', fontsize=12)
ax.set_xlabel(r'Halo Mass $M_\mathrm{halo}$ [$M_\odot$]', fontsize=12)
ax.set_title(f'SM vs HM ($z \in [{z_min},{z_max}]$)', fontsize=14)
#ax.grid(True, which="both", ls="--", alpha=0.3)
plt.legend()
plt.savefig('SM_vs_SM.png')
plt.tight_layout()
plt.show()
